In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import cupy as cp
import cudf as cd
import cuml 

In [3]:
df = cd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Id             1460 non-null   int64
 1   MSSubClass     1460 non-null   int64
 2   MSZoning       1460 non-null   object
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64
 5   Street         1460 non-null   object
 6   Alley          91 non-null     object
 7   LotShape       1460 non-null   object
 8   LandContour    1460 non-null   object
 9   Utilities      1460 non-null   object
 10  LotConfig      1460 non-null   object
 11  LandSlope      1460 non-null   object
 12  Neighborhood   1460 non-null   object
 13  Condition1     1460 non-null   object
 14  Condition2     1460 non-null   object
 15  BldgType       1460 non-null   object
 16  HouseStyle     1460 non-null   object
 17  OverallQual    1460 non-null   int64
 18  OverallCond    1460 non-null   

In [4]:
columns_needed = []
#this is done so as to choose only columns having integer and float datatype as rest of them are not needed
for col in df.columns:
    if df[col].dtype in ['int64', 'float64', 'int32', 'float32']:
        columns_needed.append(col)
final_df = df[columns_needed]

In [5]:
final_df.info()
#now cols are reduced from 81 to 38

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Id             1460 non-null   int64
 1   MSSubClass     1460 non-null   int64
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64
 4   OverallQual    1460 non-null   int64
 5   OverallCond    1460 non-null   int64
 6   YearBuilt      1460 non-null   int64
 7   YearRemodAdd   1460 non-null   int64
 8   MasVnrArea     1452 non-null   float64
 9   BsmtFinSF1     1460 non-null   int64
 10  BsmtFinSF2     1460 non-null   int64
 11  BsmtUnfSF      1460 non-null   int64
 12  TotalBsmtSF    1460 non-null   int64
 13  1stFlrSF       1460 non-null   int64
 14  2ndFlrSF       1460 non-null   int64
 15  LowQualFinSF   1460 non-null   int64
 16  GrLivArea      1460 non-null   int64
 17  BsmtFullBath   1460 non-null   int64
 18  BsmtHalfBath   1460 non-null   int64
 19  

In [6]:
#filling null values
final_df.fillna(final_df.mean(),inplace=True)


In [7]:
final_df.isnull().sum()

Id               0
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SalePrice        0
dtype: uint64

In [8]:
#indep and dep var
X = final_df.iloc[:, 1:-1]
Y = final_df.iloc[:,-1]

In [9]:
X.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,548,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,460,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,608,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,642,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,836,192,84,0,0,0,0,0,12,2008


In [10]:
Y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [11]:
#finding moderate to highly skewed feature so as to remove high deviation of values from cols
skewed_feature = [col for col in X.columns if abs(X[col].skew())>0.5]
len(skewed_feature)

29

In [12]:
skewed_feature

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'HalfBath',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [13]:
#here I used log method as it is used for higly and moderately skewed cols also whereas other methods like sqrt method works for moderately skewed method only

for feature in skewed_feature:
    X[feature] = cp.log1p(X[feature])


In [14]:
#Mean-centring the features
from cuml.preprocessing import StandardScaler
std = StandardScaler()
X = std.fit_transform(X)

In [15]:
from cuml.preprocessing import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.9, random_state=29)

In [16]:
#we will make a function which will calculate mean squared error,mean absolute error and r2 score for each of the five models
from cuml.metrics.regression import r2_score, mean_absolute_error, mean_squared_error
def get_accuracy_check(model,model_name,x_test,y_test):
    y_predict = model.predict(x_test)
    y_test = y_test.astype('float64')
    mse =  mean_squared_error(y_test,y_predict)
    mae = mean_absolute_error(y_test,y_predict)
    r2 = r2_score(y_test,y_predict)
    print("MSE for ",model_name, " is ",mse)
    print("MAE for ",model_name, " is ", mae)
    print("R2_score for ",model_name, " is ", r2)
    return [mse, mae, r2] 


In [17]:
from cuml.linear_model import LinearRegression
algos = ["eig", 'qr', "svd", "svd-jacobi", "svd-qr"]
metrics = []
models = []
for algo in algos:
    lr = LinearRegression(algorithm=algo)
    %time lr.fit(X_train, Y_train)
    models.append(lr)
    metrics.append(get_accuracy_check(lr, algo, X_test, Y_test))

CPU times: user 528 ms, sys: 289 ms, total: 817 ms
Wall time: 2.18 s
MSE for  eig  is  717286066.245657
MAE for  eig  is  20423.08856245204
R2_score for  eig  is  0.8310247256392927
CPU times: user 4.39 ms, sys: 2.96 ms, total: 7.34 ms
Wall time: 15.6 ms
MSE for  qr  is  717286066.2456628
MAE for  qr  is  20423.088562452125
R2_score for  qr  is  0.8310247256392914
CPU times: user 9.67 ms, sys: 0 ns, total: 9.67 ms
Wall time: 16.4 ms
MSE for  svd  is  717286066.2456627
MAE for  svd  is  20423.08856245212
R2_score for  svd  is  0.8310247256392914
CPU times: user 6.74 ms, sys: 1.98 ms, total: 8.73 ms
Wall time: 8.53 ms
MSE for  svd-jacobi  is  717286066.2456627
MAE for  svd-jacobi  is  20423.08856245212
R2_score for  svd-jacobi  is  0.8310247256392914
CPU times: user 7.42 ms, sys: 1.15 ms, total: 8.57 ms
Wall time: 12.4 ms
MSE for  svd-qr  is  717286066.2456653
MAE for  svd-qr  is  20423.088562452143
R2_score for  svd-qr  is  0.8310247256392908


**Performing same Preprocessing steps on test data**
 

In [18]:
df_test = cd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [19]:
df_test.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Id             1459 non-null   int64
 1   MSSubClass     1459 non-null   int64
 2   MSZoning       1455 non-null   object
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64
 5   Street         1459 non-null   object
 6   Alley          107 non-null    object
 7   LotShape       1459 non-null   object
 8   LandContour    1459 non-null   object
 9   Utilities      1457 non-null   object
 10  LotConfig      1459 non-null   object
 11  LandSlope      1459 non-null   object
 12  Neighborhood   1459 non-null   object
 13  Condition1     1459 non-null   object
 14  Condition2     1459 non-null   object
 15  BldgType       1459 non-null   object
 16  HouseStyle     1459 non-null   object
 17  OverallQual    1459 non-null   int64
 18  OverallCond    1459 non-null   

In [20]:
#Because SalePrice Column is Absent in testing dataset
columns_needed.remove('SalePrice')

In [21]:
test = df_test[columns_needed].iloc[:, 1:]

In [22]:
test.fillna(test.mean(), inplace=True)

In [23]:
test_skew_features = [col for col in test.columns if abs(test[col].skew()) > 0.5]
len(test_skew_features)

26

In [24]:
test_skew_features

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'YearBuilt',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'HalfBath',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [25]:
for feature in test_skew_features:
    test[feature] = cp.log1p(test[feature])

In [26]:
#Mean-centring the features
X_test1 = std.fit_transform(test)
X_test1.shape

(1459, 36)

In [27]:
model_used = models[0]
Y_pred = model_used.predict(X_test1)

In [28]:
predictions = {
    'Id': df_test['Id'].astype('int32'),
    'SalePrice': Y_pred,
}
sub = cd.DataFrame(predictions)
print(sub.info())
sub


<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Id         1459 non-null   int32
 1   SalePrice  1459 non-null   float64
dtypes: float64(1), int32(1)
memory usage: 17.1 KB
None


,Id,SalePrice
0,1461,122019.535007
1,1462,167221.184528
2,1463,185760.562714
3,1464,207450.881976
4,1465,211844.303723
...,...,...
1454,2915,41503.973476
1455,2916,56371.432833
1456,2917,178542.516911
1457,2918,102974.618384


In [29]:
sub.to_csv('submission.csv', index=False)